# Simulacion de sensores de un vehiculo

In [33]:
import time
import random

### Parámetros de simulación

In [34]:
initial_speed = 0  # Velocidad inicial en km/h
max_speed = 50  # Velocidad máxima en entorno urbano (km/h)
acceleration = 5  # Aceleración en km/h por segundo
deceleration = 7  # Deceleración normal en km/h por segundo
stop_deceleration = 15  # Deceleración rápida pero gradual para detenerse (en km/h por segundo)
stop_chance = 0.1  # Probabilidad de tener que detenerse por completo (semáforo/alto)
stop_duration_range = (3, 7)  # Duración de las paradas en segundos
time_between_stops = 7  # Tiempo mínimo entre paradas en segundos
speed = initial_speed

is_accelerating = True
is_stopped = False
is_slowing_down = False
stop_timer = 0
last_stop_time = time.time()  # Inicializamos con el tiempo actual

### Simulación de velocidad

In [35]:
def simulate_speed(current_speed, max_speed, acceleration, deceleration, stop_deceleration):
    global is_accelerating, is_stopped, stop_timer, is_slowing_down, last_stop_time

    # Obtener el tiempo actual
    current_time = time.time()
    time_since_last_stop = current_time - last_stop_time

    # Si el vehículo está detenido en un semáforo o señal de alto
    if is_stopped:
        stop_timer -= 0.1  # Restamos 0.1 segundos al contador de parada
        if stop_timer <= 0:
            is_stopped = False  # Reiniciamos y el vehículo comienza a moverse
        return 0  # Mientras está detenido, la velocidad es 0

    # Simular paradas aleatorias en semáforos o señales de alto
    if current_speed > 40 and time_since_last_stop > time_between_stops and random.random() < stop_chance:  # Solo si va a más de 40 km/h y ha pasado suficiente tiempo
        print(' ======> STOPP!')
        is_slowing_down = True
        last_stop_time = current_time  # Actualizamos el último tiempo de parada
        return max(0, current_speed)  # Mantenemos la velocidad actual mientras desaceleramos

    # Si está en proceso de desaceleración para parar
    if is_slowing_down:
        new_speed = current_speed - stop_deceleration * 0.1  # Desaceleramos rápidamente pero gradualmente
        if new_speed <= 0:  # Cuando la velocidad llega a 0, el vehículo se detiene
            is_slowing_down = False
            is_stopped = True
            stop_timer = random.uniform(*stop_duration_range)  # Generar duración aleatoria de la parada
            return 0
        return round(new_speed, 2)  # Devolvemos la nueva velocidad gradualmente disminuida

    # Simular aceleración y desaceleración normales
    if is_accelerating:
        new_speed = current_speed + acceleration * 0.1  # Aceleramos
        if new_speed >= max_speed:
            new_speed = max_speed
            is_accelerating = False  # Cambiamos a desaceleración
    else:
        new_speed = current_speed - deceleration * 0.1  # Desaceleramos normalmente
        if new_speed <= 0:
            new_speed = 0
            is_accelerating = True  # Cambiamos de nuevo a aceleración

    # Añadir ruido para simular fluctuaciones realistas
    new_speed += random.uniform(-0.5, 0.5)  # Pequeño ruido entre -0.5 y 0.5 km/h
    return max(0, round(new_speed, 2))  # No permitimos que la velocidad sea negativa


## Simulación de vehiculo en movimiento

In [ ]:
try:
    while True:
        speed = simulate_speed(speed, max_speed, acceleration, deceleration, stop_deceleration)
        print(f"Velocidad simulada: {speed} km/h")

        # Pausa de 0.1 segundos para simular el tiempo de muestreo
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Simulación detenida")